<a href="https://colab.research.google.com/github/sunny0103/DeepLearning_projects/blob/main/predict_judge/Predict_Jugment_ML_stacking_kfold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* 0: loss, 1:win

In [1]:
!pip install rich

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
cd "/content/drive/MyDrive/Data/judge"

/content/drive/MyDrive/Data/judge


In [4]:
import warnings
warnings.filterwarnings('ignore')

import os, sys
import rich
from rich.table import Table
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
# from sklearn.linear_model import SGDClassifier
# from skelarn.ensemble import RandomForestClassifier, AdaBoostClassifier
# from sklearn.svm import SVC
# from sklearn.neighbors import KNeighborsClassifier

from sklearn.ensemble import StackingClassifier
from tqdm.auto import tqdm
from sklearn.linear_model  import LogisticRegression
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import (RandomForestClassifier,
                              AdaBoostClassifier,
                              ExtraTreesClassifier,
                              BaggingClassifier,
                              GradientBoostingClassifier,
                              VotingClassifier)

from sklearn.neighbors import KNeighborsClassifier
from lightgbm import LGBMClassifier


# import nltk
# nltk.download('all')
# from nltk.corpus import stopwords

In [5]:
train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')
print('--train--')
print(train_df.shape)
display(train_df.head())
print('--test--')
print(test_df.shape)
display(test_df.head())

--train--
(2478, 5)


,ID,first_party,second_party,facts,first_party_winner
0,TRAIN_0000,Phil A. St. Amant,Herman A. Thompson,"On June 27, 1962, Phil St. Amant, a candidate ...",1
1,TRAIN_0001,Stephen Duncan,Lawrence Owens,Ramon Nelson was riding his bike when he suffe...,0
2,TRAIN_0002,Billy Joe Magwood,"Tony Patterson, Warden, et al.",An Alabama state court convicted Billy Joe Mag...,1
3,TRAIN_0003,Linkletter,Walker,Victor Linkletter was convicted in state court...,0
4,TRAIN_0004,William Earl Fikes,Alabama,"On April 24, 1953 in Selma, Alabama, an intrud...",1


--test--
(1240, 4)


,ID,first_party,second_party,facts
0,TEST_0000,Salerno,United States,The 1984 Bail Reform Act allowed the federal c...
1,TEST_0001,Milberg Weiss Bershad Hynes and Lerach,"Lexecon, Inc.",Lexecon Inc. was a defendant in a class action...
2,TEST_0002,No. 07-582\t Title: \t Federal Communications ...,"Fox Television Stations, Inc., et al.","In 2002 and 2003, Fox Television Stations broa..."
3,TEST_0003,Harold Kaufman,United States,During his trial for armed robbery of a federa...
4,TEST_0004,Berger,Hanlon,"In 1993, a magistrate judge issued a warrant a..."


In [6]:
# drop_col = ['first_party', 'second_party']
# train_df = train_df.drop(drop_col, axis=1)
# test_df = test_df.drop(drop_col, axis=1)

In [7]:
# train_df.isnull().sum()

In [8]:
# train_text = list(train_df['facts'])
# test_text = list(test_df['facts'])

In [9]:
# from nltk.stem import WordNetLemmatizer
# lemmatizer = WordNetLemmatizer() #표제어 추출

In [10]:
# def create_corpus(text):
#   corpus =[]

#   for i in range(len(text)):
#     r = re.sub('[^a-zA-Z]', ' ', text[i])
#     r = r.lower()
#     r = r.split()
#     r = [word for word in r if word not in stopwords.words('english')]
#     r = [lemmatizer.lemmatize(word) for word in r]
#     r = ' '.join(r)
#     corpus.append(r)

#   return corpus

def preprocess(text):
  text = text.lower()
  text = re.sub('[^a-zA-Z]', ' ', text)  # re.sub(r"[^A-Za-zㄱ-ㅎㅏ-ㅣ가-힣 ]","", text)
  return text

In [11]:
# train_df['text'] = create_corpus(train_text)
# test_df['text'] = create_corpus(test_text)

train_df['preprocessed'] = train_df.facts.apply(lambda x : preprocess(x))
test_df['preprocessed'] = test_df.facts.apply(lambda x : preprocess(x))

In [12]:
train_df['context'] = train_df['first_party'] +','+train_df['second_party'] +','+ train_df['preprocessed']
test_df['context'] = test_df['first_party'] +','+test_df['second_party'] +','+ test_df['preprocessed']

In [13]:
# vectorizer = TfidfVectorizer()
# def get_vector(vectorizer, df, train_mode):
#     if train_mode:
#         X_facts = vectorizer.fit_transform(df['text'])
#     else:
#         X_facts = vectorizer.transform(df['text'])
#     X_party1 = vectorizer.transform(df['first_party'])
#     X_party2 = vectorizer.transform(df['second_party'])

#     X = np.concatenate([X_party1.todense(), X_party2.todense(), X_facts.todense()], axis=1)
#     return X

In [14]:
# X = get_vector(vectorizer, train_df, True)
# y = train_df["first_party_winner"]
# X_test = get_vector(vectorizer, test_df, False)

In [16]:
from sklearn.pipeline import Pipeline
def get_pipe(model, model_name: str) -> Pipeline:
    "TfidfVectorizer와 모델을 연결한 파이프라인을 반환하는 함수"
    tfidf = TfidfVectorizer(analyzer="char", ngram_range=(1, 3))
    pipe = Pipeline([
        ("tfidf", tfidf),
        (model_name, model)
    ])
    return pipe

In [30]:
def return_kfold_accuarcy(model, k: int = 5) -> float:
    "모델을 입력받아 KFold 예측 후 accuracy score를 반환하는 함수"
    kfold = StratifiedKFold(k, shuffle=True, random_state=42)
    result = []
    for train_idx, test_idx in kfold.split(train_df["context"], train_df["first_party_winner"]):
        train, val = train_df.iloc[train_idx], train_df.iloc[test_idx]
        model.fit(train["context"], train["first_party_winner"])
        pred = model.predict(val["context"])
        acc = accuracy_score(val["first_party_winner"], pred)
        result.append(acc)

    return np.mean(result)

In [25]:
# from sklearn.model_selection import train_test_split

# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# print('training data:', X_train.shape)
# print('validation data:', X_val.shape)

In [26]:
# def return_kfold_accuracy(model,k:int=5)->float:
#   kfold = StratifiedKFold(k, shuffle=True, random_state=42)
#   result = []
#   for train_idx, test_idx in kfold.split(train_df['text'], train_df["first_party_winner"]):
#     train, val = train_df.loc[train_idx], train_df.loc[test_idx]
#     X_train = get_vector(vectorizer, train, True)
#     X_train = np.array(X_train)
#     X_test = get_vector(vectorizer, val, False)
#     X_test = np.array(X_test)
#     model.fit(X_train, train["first_party_winner"])
#     pred = model.predict(X_test)
#     acc = accuracy_score(val["first_party_winner"], pred)
#     result.append(acc)

#   return np.mean(result)

In [20]:
# X_train = np.array(X_train)
# X_val = np.array(X_val)

# def return_accuracy(model)->float:
#   model.fit(X_train, y_train)
#   pred = model.predict(X_val)
#   acc = accuracy_score(y_val, pred)
#   return acc

In [27]:
models =[
    # ('lr', LogisticRegression()),
    ('clf',svm.SVC(probability=True)),
    # ('gnb', GaussianNB()),
    ('rf',RandomForestClassifier()),
    # ('ada', AdaBoostClassifier()),
    ('et', ExtraTreesClassifier()),
    # ('bag', BaggingClassifier()),
    # ('grad', GradientBoostingClassifier()),
    # ('lgb', LGBMClassifier()),
    # ('kn', KNeighborsClassifier())
    ]

model_pipes = [(name, get_pipe(model, name)) for name, model in models]

In [28]:
table = Table(title = 'Model comparison')
table.add_column("Model Name", justify="left", style="green")
table.add_column("Accuracy", justify="right")


for model_name, model in tqdm(model_pipes, leave=False):
    acc = return_kfold_accuarcy(model)
    table.add_row(model_name, f"{acc:0.3f}")

rich.print(table)


  0%|          | 0/3 [00:00<?, ?it/s]

    Model comparison     
┏━━━━━━━━━━━━┳━━━━━━━━━━┓
┃ Model Name ┃ Accuracy ┃
┡━━━━━━━━━━━━╇━━━━━━━━━━┩
│ clf        │    0.665 │
│ rf         │    0.661 │
│ et         │    0.663 │
└────────────┴──────────┘

In [32]:
stack_models = [(name, get_pipe(model, name)) for name, model in models]
stacking = StackingClassifier(stack_models)
acc = return_kfold_accuarcy(stacking)
rich.print(acc)

0.6650521342456825

In [33]:
# X = np.array(X)
stacking.fit(train_df["context"], train_df["first_party_winner"])

# X_test = np.array(X_test)
y_pred = stacking.predict(test_df['context'])

In [34]:
y_pred

array([1, 1, 1, ..., 1, 1, 1])

In [35]:
submit = pd.read_csv('./sample_submission.csv')
submit['first_party_winner'] = y_pred
submit.to_csv('./stacking_fold_submit.csv', index=False)
print('Done')

Done
